In [1]:
import torch
import time
from prettytable import PrettyTable

### 3.1 Подготовка данных

In [ ]:
# Создайте большие матрицы размеров:
# - 64 x 1024 x 1024
# - 128 x 512 x 512
# - 256 x 256 x 256
# Заполните их случайными числами

In [2]:
tensor_1 = torch.rand((64, 1024, 1024))
tensor_2 = torch.rand((128, 512, 512))
tensor_3 = torch.rand((256, 256, 256))
tensors = [tensor_1, tensor_2, tensor_3]
tensor_names = ['64 x 1024 x 1024','128 x 512 x 512','256 x 256 x 256']

### 3.2 Функция измерения времени

In [ ]:
# Создайте функцию для измерения времени выполнения операций
# Используйте torch.cuda.Event() для точного измерения на GPU
# Используйте time.time() для измерения на CPU

Так как я использую Mac с чипом M1. Я буду использовать GPU - ускорение от Apple через mps

In [3]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Используемое устройство: {device}")

Используемое устройство: mps


In [4]:
def measure_time(operation, device='cpu'):
    start = time.time()
    operation()
    if device == 'mps':
        torch.mps.synchronize()
    return (time.time() - start) * 1000  # мс

### 3.3 Сравнение операций

In [ ]:
# Сравните время выполнения следующих операций на CPU и CUDA:
# - Матричное умножение (torch.matmul)
# - Поэлементное сложение
# - Поэлементное умножение
# - Транспонирование
# - Вычисление суммы всех элементов

# Для каждой операции:
# 1. Измерьте время на CPU
# 2. Измерьте время на GPU (если доступен)
# 3. Вычислите ускорение (speedup)
# 4. Выведите результаты в табличном виде

In [5]:
operations = {
    "Матричное умножение": lambda x, y: torch.matmul(x, x.transpose(-2, -1)),
    "Поэлементное сложение": lambda x, y: x + x,
    "Поэлементное умножение": lambda x, y: x * x,
    "Транспонирование": lambda x, y: x.transpose(-2, -1),
    "Сумма всех элементов": lambda x, y: x.sum()
}

table = PrettyTable()
table.field_names = ["Операция", "Размер тензора", "CPU (мс)", "GPU (мс)", "Ускорение"]

for op_name, op in operations.items():
    for tensor, name in zip(tensors, tensor_names):
        try:
            # Измеряем время на CPU
            x_cpu = tensor.clone()
            cpu_time = measure_time(lambda: op(x_cpu, None), 'cpu')
            
            # Измеряем время на GPU (MPS)
            if device.type == 'mps':
                x_gpu = x_cpu.to(device)
                gpu_time = measure_time(lambda: op(x_gpu, None), 'mps')
                speedup = cpu_time / gpu_time
            else:
                gpu_time = "N/A"
                speedup = "N/A"
            
            table.add_row([
                op_name, 
                name, 
                f"{cpu_time:.2f}", 
                f"{gpu_time:.2f}" if device.type == 'mps' else gpu_time, 
                f"{speedup:.2f}x" if device.type == 'mps' else speedup
            ])
        except Exception as e:
            print(f"Ошибка при выполнении '{op_name}' с тензором {name}: {str(e)}")
            table.add_row([op_name, name, "Ошибка", "Ошибка", "Ошибка"])

print(table)

+------------------------+------------------+----------+----------+-----------+
|        Операция        |  Размер тензора  | CPU (мс) | GPU (мс) | Ускорение |
+------------------------+------------------+----------+----------+-----------+
|  Матричное умножение   | 64 x 1024 x 1024 |  456.80  |  624.82  |   0.73x   |
|  Матричное умножение   | 128 x 512 x 512  |  104.62  |  64.22   |   1.63x   |
|  Матричное умножение   | 256 x 256 x 256  |  28.53   |   9.81   |   2.91x   |
| Поэлементное сложение  | 64 x 1024 x 1024 |  197.32  |  55.16   |   3.58x   |
| Поэлементное сложение  | 128 x 512 x 512  |  11.56   |   9.84   |   1.17x   |
| Поэлементное сложение  | 256 x 256 x 256  |   5.87   |   5.71   |   1.03x   |
| Поэлементное умножение | 64 x 1024 x 1024 |  128.24  |  38.80   |   3.30x   |
| Поэлементное умножение | 128 x 512 x 512  |  10.74   |   8.12   |   1.32x   |
| Поэлементное умножение | 256 x 256 x 256  |   5.68   |   5.86   |   0.97x   |
|    Транспонирование    | 64 x 1024 x 1

### 3.4 Анализ результатов

В HOMEWORK.md